In [1]:
# Importing all the required library
from bs4 import BeautifulSoup
import urllib.request, urllib.error
import psycopg2
from flask import Flask, render_template
import json
import time
import schedule

In [2]:
def data_scraping():
    # Performing step 1 of the group project
    # scarping data from website "espn"
    # from scrapin we have used Beautiful soup
    print("gathering information")
    # url of the website
    url = "https://www.espncricinfo.com/records/trophy/batting-most-runs-career/world-cup-12"
    # requesting for url and read the url
    html = urllib.request.urlopen(url).read()
    # creating soup object of html parsed data from website
    soup = BeautifulSoup(html, "html.parser")
    # finding for the mentioned class for div tag
    data = soup.find("div", class_="ds-overflow-x-auto ds-scrollbar-hide")
    # finding all tr tag which contains actual data of the cricketer and it will return a list
    # escaping 0 element because it contains heading 
    data01 = data.find_all("tr")[1:]
    # creating emplty list to store scraped data 
    cricketers = []
    # looping thorugh list which we got form data01
    for cricketer in data01:
        # all the players statistics and name is under td tag.
        players_data = cricketer.find_all("td")
        # capturing data from players_data using indexig as each dat ais stored under different td tags
        player_name = players_data[0].text
        years_played = players_data[1].text
        matches_played = players_data[2].text
        innings_played = players_data[3].text
        batting_position = players_data[4].text
        total_runs_scored = players_data[5].text
        highest_score = players_data[6].text
        average = players_data[7].text
        strike_rate = players_data[9].text
        # manually entering value 0 which is integer to '-' which is character meaning of both is same just different data type
        if ((players_data[10].text) == '-'):
            century = 0
        else:    
            century = players_data[10].text

         # manually entering value 0 which is integer to '-' which is character meaning of both is same just different data type
        if ((players_data[11].text) == '-'):
            half_century = 0
        else:    
            half_century = players_data[11].text

        # appending gathered data to created empty list
        cricketers.append([player_name, years_played, matches_played, innings_played, batting_position, total_runs_scored,
                          highest_score, average, strike_rate, century, half_century])

    # Step 2
    # uploading scaped data to databse
    # we have used Postgress SQL RDBMS to store data.
    # following are the details of the postgress which is uded to connect to datbase
    hostname = 'localhost'
    database = 'Final_project'
    username = 'postgres'
    pwd = 'Nihar@0704'
    port_id = 5432

    # establising connection to datbase
    conn = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)

    # creating cursor
    cur = conn.cursor()

    # exwcuting statment under the brackets ()
    cur.execute('DROP TABLE IF EXISTS cricketer')

    #defing table schema
    table = ''' CREATE TABLE IF NOT EXISTS cricketer (
                    id            SERIAL PRIMARY KEY,
                    Player_Name   varchar(50) NOT NULL,
                    Years_played  varchar(50) NOT NULL,
                    Matches       int,
                    Innings       int,
                    Bat_Position  int,
                    Total_Runs    int,
                    Highest_Score varchar(20),
                    Average       decimal,
                    Srike_Rate    decimal,
                    Century       int,
                    Half_Century  int) '''

    # executing tabel schema which will create a table named cricketer
    cur.execute(table)

    # sql statment to insert the data into table
    insert_data = '''INSERT INTO cricketer (Player_Name, Years_played, Matches, Innings, Bat_Position, Total_Runs, Highest_Score,
                                        Average, Srike_Rate, Century, Half_Century) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

    # inserting data of each cricketer frim cricketers list using insert_data object and executing it to databse
    for table_data in cricketers:
        cur.execute(insert_data, table_data)

    # commiting chages to SQL datbase
    conn.commit()
    # closing the currsor which is good practice
    cur.close()
    # similar
    conn.close()




gathering information


In [3]:
schedule.every(24).hours.do(data_scraping)


In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

In [8]:
%%writefile Final_Group_project.py

from flask import Flask, render_template
import psycopg2

# Flask application setup
app = Flask(__name__)

# Function to fetch data from the PostgreSQL database
def fetch_data():
    hostname = 'localhost'
    database = 'Final_project'
    username = 'postgres'
    pwd = 'Nihar@0704'
    port_id = 5432

    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id
    )

    cur = conn.cursor()

    cur.execute('SELECT * FROM cricketer')
    data = cur.fetchall()

    cur.close()
    conn.close()

    return data


# Route to display the dashboard
@app.route('/')
def display_dashboard():
    cricketer_data = fetch_data()

    # Prepare data for Google Charts (Total Runs)
    chart_data_runs = "[['Player Name', 'Total Runs'],"
    for player in cricketer_data:
        player_name = player[1].replace("'", "\\'")  # Escape single quotes
        # concatenate the chart_data_runs sring
        # add total runs which is at 6th index
        chart_data_runs += "['{}', {}],".format(player_name, player[6])
    chart_data_runs += "]"

    # Prepare data for Google Charts Top 5 Centuries
    # sort the list with the century which is at 10th index and will sort for the first 5 player with highest numberof century
    top5_centuries = sorted(cricketer_data, key=lambda x: x[10], reverse=True)[:5]
    chart_data_top5_centuries = "[['Player Name', 'Centuries'],"
    for player in top5_centuries:
        player_name = player[1].replace("'", "\\'") 
        chart_data_top5_centuries += "['{}', {}],".format(player_name, player[10])
    chart_data_top5_centuries += "]"

    # Prepare data for Google Charts (Top 3 Strike Rates)
    top3_strike_rates = sorted(cricketer_data, key=lambda x: x[9], reverse=True)[:3]
    chart_data_top3_strike_rates = "[['Player Name', 'Strike Rate'],"
    for player in top3_strike_rates:
        player_name = player[1].replace("'", "\\'") 
        chart_data_top3_strike_rates += "['{}', {}],".format(player_name, player[9])
    chart_data_top3_strike_rates += "]"

    # Display the HTML and JavaScript code with the charts
    return render_template('dashboard.html', chart_data_runs=chart_data_runs, chart_data_top5_centuries=chart_data_top5_centuries, chart_data_top3_strike_rates=chart_data_top3_strike_rates)


if __name__ == '__main__':
    app.run(debug=True)


Overwriting Final_Group_project.py


In [9]:
%%writefile templates/dashboard.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Dashboard</title>
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
</head>
<body>
    <div id="chart_div_runs" style="width: 80%; height: 400px; margin: auto;"></div>
    <div id="chart_div_top5_centuries" style="width: 80%; height: 400px; margin: auto;"></div>
    <div id="chart_div_top3_strike_rates" style="width: 80%; height: 400px; margin: auto;"></div>

    <script>
        google.charts.load('current', {'packages':['corechart']});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
            // Draw Total Runs Chart
        var dataRuns = google.visualization.arrayToDataTable({{ chart_data_runs | safe }});

        var optionsRuns = {
          title: 'Total Runs by Cricketers',
          hAxis: {title: 'Player Name', titleTextStyle: {color: '#333'}},
          vAxis: {minValue: 0}
        };

        var chartRuns = new google.visualization.ColumnChart(document.getElementById('chart_div_runs'));
        chartRuns.draw(dataRuns, optionsRuns);

        // Draw Top 5 Centuries Pie Chart
        var dataTop5Centuries = google.visualization.arrayToDataTable({{ chart_data_top5_centuries | safe }});

        var optionsTop5Centuries = {
          title: 'Top 5 Players with Most Centuries',
        };

        var chartTop5Centuries = new google.visualization.PieChart(document.getElementById('chart_div_top5_centuries'));
        chartTop5Centuries.draw(dataTop5Centuries, optionsTop5Centuries);

        // Draw Top 3 Strike Rates Chart
        var dataTop3StrikeRates = google.visualization.arrayToDataTable({{ chart_data_top3_strike_rates | safe }});

        var optionsTop3StrikeRates = {
          title: 'Top 3 Players with Highest Strike Rates',
          hAxis: {title: 'Player Name', titleTextStyle: {color: '#333'}},
          vAxis: {minValue: 0}
        };

        var chartTop3StrikeRates = new google.visualization.ColumnChart(document.getElementById('chart_div_top3_strike_rates'));
        chartTop3StrikeRates.draw(dataTop3StrikeRates, optionsTop3StrikeRates);
      }
    </script>
</body>
</html>


Overwriting templates/dashboard.html
